Extração e Salvamento de Dados da API Brasil no Banco de Dados

In [3]:
import sqlite3
import requests
import pandas as pd
from plyer import notification

def extract_data(api_url):
    """
    Extrai dados de uma API através de uma solicitação GET.

    Args:
        api_url (str): A URL da API.

    Returns:
        dict or None: Os dados extraídos da API em formato JSON, ou None se a requisição falhar.
    """
    # Faz uma requisição GET na URL
    req = requests.get(api_url)

    # Verifica o status da requisição
    if req.status_code == 200:
        # Converte a resposta para o formato JSON
        data = req.json()
        return data
    else:
        return None

def process_banks_data(data):
    """
    Processa os dados dos bancos obtidos da API.

    Args:
        data (dict): Os dados brutos obtidos da API.

    Returns:
        pandas.DataFrame or None: Um DataFrame contendo os dados processados dos bancos, ou None se os dados brutos forem None.
    """
    if data:
        # Cria um DataFrame com os dados da API
        df = pd.DataFrame(data)
        
        # Renomeia as colunas do DataFrame
        df.rename(columns={'ispb': 'ISPB', 'name': 'Nome', 'code': 'Código', 'fullName': 'Nome Banco'}, inplace=True)

        # Converte a coluna 'Código' que estava em float para o tipo inteiro
        df['Código'] = df['Código'].astype('Int64')

        # Preenche valores ausentes na coluna 'Código' de <NA> com zero 
        df.fillna({'Código': 0}, inplace=True)

        # Transforma todos os valores na coluna 'Nome' em maiúsculas
        df['Nome'] = df['Nome'].str.upper()

        # Transforma a primeira letra de cada palavra na coluna 'Nome Banco' em maiúscula
        df['Nome Banco'] = df['Nome Banco'].str.title()
        
        return df
    else:
        return None

def process_cities_data(data):
    """
    Processa os dados das cidades obtidos da API.

    Args:
        data (dict): Os dados brutos obtidos da API.

    Returns:
        pandas.DataFrame or None: Um DataFrame contendo os dados processados das cidades, ou None se os dados brutos forem None.
    """
    if data:
        # Cria um DataFrame com os dados da API
        df = pd.DataFrame(data)

        # Renomeia as colunas do DataFrame
        df.rename(columns={'nome': 'Cidade','id':'Código', 'estado': 'UF'}, inplace=True)

        return df
    else:
        return None

def process_states_data(data):
    """
    Processa os dados dos estados obtidos da API.

    Args:
        data (dict): Os dados brutos obtidos da API.

    Returns:
        pandas.DataFrame or None: Um DataFrame contendo os dados processados dos estados, ou None se os dados brutos forem None.
    """
    if data:
        # Cria um DataFrame com os dados da API
        df = pd.DataFrame(data)

        # Tratativa dos campos em conjunto da API anteriormente {"id":1,"sigla":"N","nome":"Norte"},
        # Ação necessaria para trazer em colunas conforme está no dataframe regiao_id/regiao_sigla/regiao_nome
        df['regiao_id'] = df['regiao'].apply(lambda id: id['id'])
        df['regiao_sigla'] = df['regiao'].apply(lambda sigla: sigla['sigla'])
        df['regiao_nome'] = df['regiao'].apply(lambda nome: nome['nome'])

        # Dropar a coluna 'regiao' original
        df.drop(columns=['regiao'], inplace=True)

        # Renomear as colunas do DataFrame
        df.rename(columns={'id': 'Código', 'sigla': 'UF', 'nome': 'Estado'}, inplace=True)

        return df
    else:
        return None

def save_to_database(df, table_name):
    """
    Salva um DataFrame em uma tabela do banco de dados SQLite.

    Args:
        df (pandas.DataFrame): O DataFrame a ser salvo.
        table_name (str): O nome da tabela onde os dados serão salvos.
    """
    if df is not None:
        # Ligação ao Banco de Dados 
        conn = sqlite3.connect('projetofinal.db')
        
        # Salva o DataFrame no SQLite
        df.to_sql(table_name, conn, if_exists='replace', index=False)

        # Alerta de sucesso
        notification.notify(
            title='Sucesso',
            message=f'Dados salvos na tabela {table_name} com sucesso!',
            app_name='Alerta',
            timeout=10
        )

        # Fecha conexão com banco de dados
        conn.close()
    else:
        # Alerta de erro
        notification.notify(
            title='Erro',
            message=f'Falha ao salvar dados na tabela {table_name}!',
            app_name='Alerta',
            timeout=10
        )

def extract_and_save_to_db(api_url, table_name, process_func):
    """
    Extrai dados de uma API, processa-os e salva-os em uma tabela do banco de dados.

    Args:
        api_url (str): A URL da API.
        table_name (str): O nome da tabela onde os dados serão salvos.
        process_func (function): A função para processar os dados extraídos da API.
    """
    # Extrai os dados da API
    data = extract_data(api_url)
    # Processa os dados extraídos usando a função especificada
    processed_data = process_func(data)
    # Salva os dados processados na tabela do banco de dados
    save_to_database(processed_data, table_name)

# Chamada de utilização para a primeira API
api_url_1 = "https://brasilapi.com.br/api/banks/v1"
table_name_1 = 'bancos'
extract_and_save_to_db(api_url_1, table_name_1, process_banks_data)

# Chamada de utilização para a segunda API
api_url_2 = "https://brasilapi.com.br/api/cptec/v1/cidade"
table_name_2 = 'cidades'
extract_and_save_to_db(api_url_2, table_name_2, process_cities_data)

# Chamada de utilização para a terceira API
api_url_3 = "https://brasilapi.com.br/api/ibge/uf/v1"
table_name_3 = 'estados'
extract_and_save_to_db(api_url_3, table_name_3, process_states_data)
